<a href="https://www.kaggle.com/code/yaaangzhou/very-simple-models-4-breast-cancer-clasification?scriptVersionId=135619906" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer-wisconsin-diagnostic-dataset/brca.csv


# Import Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')



In [3]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-diagnostic-dataset/brca.csv', index_col=0)

# Data Preprocessing

In [4]:
var_obj = df['y'].map({'B': 0, 'M': 1})
var_imp = df.drop('y', axis=1)

In [5]:
var_imp_scala = StandardScaler().fit_transform(var_imp)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(var_imp_scala, var_obj, test_size=0.3, random_state=42)

# Select Models

## RF

In [7]:
model_rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100,  300, 500],
    'max_depth': [3, 5, 8, 10],
    'min_samples_split': [2, 3, 5, 6],
    'min_samples_leaf': [1, 3, 5],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [8]:
model_rf_cv = GridSearchCV(model_rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
model_rf_cv.fit(x_train, y_train)


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 8, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 3, 5, 6],
                         'n_estimators': [100, 300, 500]},
             scoring='accuracy')

## XGBoost

In [9]:
model_xgb = XGBClassifier()

In [10]:
param_grid = {
    "max_depth": [1, 3, 5, 7],
    "learning_rate": [0.5, 0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1, 3],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.7],
    "colsample_bytree": [0.5],
}

In [11]:
model_xgb_cv = GridSearchCV(model_xgb, param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=0)
model_xgb_cv.fit(x_train, y_train)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.5], 'gamma': [0, 0.25, 1, 3],
                         'learning_rate': [0.5, 0.1, 0.01, 0.05],
                         'max_depth': [1, 3, 5, 7], 'reg_lambda': [0, 1, 10],
                         'scale_pos_weight': [1, 3, 5], 'subsample': [0.7]},
             scoring='roc_auc')

# Prediction

In [12]:
y_pred_rf = model_rf_cv.predict(x_test)
y_pred_xgb = model_xgb_cv.predict(x_test)

print('model random forest',accuracy_score(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))
print('model xgb',accuracy_score(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

model random forest 0.9415204678362573
[[105   3]
 [  7  56]]
model xgb 0.935672514619883
[[102   6]
 [  5  58]]


As a simple try, the rf model already has an accuracy score with 0.94, and the xgb model has an accuracy score with 0.93.

It's not bad, but if u want to make it better, try stacking/bagging o sth else.